In [ ]:
import pandas as pd
import polars as pl

import sys
sys.path.insert(1, '../framework')
from racetrack      import *
from rt_panel_mixin import RTGraphInteractiveLayout

rt = RACETrack()

df = pl.DataFrame({'fm':'a b c d e a'.split(),
                   'to':'b c d e a d'.split()})
_pos_    = {'a':(0.0,0.0), 'b':(1.0,0.0), 'c':(1.0,1.0), 'd':(0.5,1.0), 'e':(0.0,1.0)}
_params_ = {'relationships':[('fm','to')], 'draw_labels':True, 'render_pos_context':True, 
            'link_size':'large', 'link_opacity':0.2, 'pos':_pos_}
_rtg_ = rt.interactiveGraphLayout(df, _params_, w=300, h=200)
_rtg_

In [ ]:
#_rtg_.info_str = "Test"
#_rtg_.selected_entities, _rtg_.selectionpath, _rtg_.last_key, _rtg_.allentities_x0, _rtg_.allentities_y0
_rtg_.selectEntities(set(['a','b','c']))
_rtg_.selectedEntities()

In [ ]:
_rtg_.mod_inner
#rt.linkNode(df, pos=_pos_, **_params_)

In [ ]:
df_nf  = pl.read_csv('../../netflow_sample.csv').sample(10_000)
print(len(set(df_nf['sip']) | set(df_nf['dip'])))
pos_nf = {}
print('total nodes = ', len(set(df_nf['sip']) | set(df_nf['dip'])))

In [ ]:
#_rtg_nf_ = rt.interactiveGraphLayout(df_nf, {'relationships':[('sip','dip')], 'draw_labels':True, 'link_opacity':0.1, 'pos':pos_nf})
#_rtg_nf_

In [ ]:
# ... so only keys that are safe with shift will be used // 2024-06-16
# in firefox for jupyter lab, neither "m" nor "r" are safe to use (unless defaults are prevented... then everything
# is safe (assuming w/out control being pressed) ... except that's not how visual studio code works...
# okay "  c e ghi   mn pqrstu w yz" -- these keys don't seem to be shortcuts // visual studio code
# okay "  c e ghi    n pq stu w yz" -- these keys don't seem to be shortcuts // firefox for jupyter lab
# used "  c e gh         q stu w  z"
# aval "        i    n p          " -- set(okay) minus set(used)
# ... all are safe with "shift" ... any mouse does is not safe with shift 2024-06-18

# 'c' :            - layout selection (CIRCLE)
# 'e' :            - expand selection 
# 'e' : shift      - use a digraph
# 'g' :            - layout selection (grid)
# 'h' :            - layout selection (SUNFLOWER)
# 'q' :            - invert selection
# 'q' : shift      - select common neighbors
# 's' :            - set selected nodes to the sticky label set
# 's' : shift      - subtract nodes from sticky labels
# 't':             - combine into a single node
# 'u' :            - layout selection (line)
# 'w' :            - add nodes to sticky labels
# 'w' : shift      - toggle through "all labels" / "sticky labels" / "no labels"
# 'y':             - vertical line of nodes (selected)
# 'y':  shift      - horizontal line of nodes (selected)
# 'z' :            - zoom (if selected nodes, fit all selected in view) ... if none selected, fit all nodes in the view
# 'z' : shift      - selected w/ all neighbors in view

# 'spc':           - remove nodes from dataframe (push new dataframe onto the stack)
# 'spc': shift     - add them back in (go up the stack)
# 'spc': ctrl      - go down the stack (what's already on the stack)

# '1..90':         - select nodes by degree w/ 0 == > 9 degree
# '1..90':         - (if nodes are already selected... do the intersection with those nodes)
# '1..90': shift   - subtract those nodes from the selection

#
# OLD KEYBOARD SHORTCUTS (used ctrl) // prior to 2024-06-16
#

# ... that leaves "cehimstuyz" as the only safe keys for shift & control combinations
# ... used so far "ce   st  z"
# ... 1..9 are all safe... shift + 1..9 are all safe... but not ctrl + 1..9
# ... usually - "shift" is subtract, "ctrl" is add, and "shift-ctrl" is intersect

## LAYOUT
# DONE 'c': grid       (most common use)
# DONE 'c': shift      - line
# DONE 'c': ctrl       - sunflower
# DONE 'c': ctrl-shift - circle
# DONE 't': shift      - horizontal (or vertical)
# DONE 't': ctrl       - vertical (or horizontal)
# DONE 't': collapse to a single point

## SHORTCUTS
# DONE 'e' :                 - expand selection 
# DONE 'e' : ctrl            - use a digraph
# DONE 'e' : shift           - invert selection
# DONE 'e' : ctrl-shift      - select common neighbors
# DONE 's' :                 - set selected nodes to the sticky label set
# DONE 's' : ctrl            - add nodes to sticky labels
# DONE 's' : shift           - subtract nodes from sticky labels
# DONE 's' : ctrl-shift      - toggle through "all labels" / "sticky labels" / "no labels"
# DONE 'z' :                 - zoom (if selected nodes, fit all selected in view) ... if none selected, fit all nodes in the view
# DONE 'z' : shift           - reset to all nodes in view (if you wanted to just keep your selection)
# DONE 'z' : ctrl-shift      - selected w/ all neighbors in view
# DONE:'spc':                - remove nodes from dataframe (push new dataframe onto the stack)
# DONE:'spc': shift          - add them back in (go up the stack)
# DONE:'spc': ctrl           - go down the stack (what's already on the stack)

## INTERACTION (Dragging w/ the Mouse)
# DONE : select
# DONE : ctrl       - add to select
# DONE : shift      - subtract from select
# DONE : ctrl-shift - intersect with select

# ???     : select color under the mouse / shift-subtract / ctrl-add
# '1..90' : select node by degree 1...9 ... 0 is degree 10 or more // only shift is safe... can't use control


In [ ]:
df = pl.DataFrame({'fm':'a b c d'.split(), 'to':'b c d a'.split(), 'ct':[1, 1, 6, 3]})
_pos_ = {'a':(0.0,0.0), 'b':(1.0,0.0), 'c':(1.0,1.0), 'd':(0.5,1.0)}
_params_ = {'relationships':[('fm','to')], 'pos':_pos_, 'link_size':'vary', 'count_by':'ct', 
            'draw_labels':True, 'node_labels':{'d':'test','c':'more'}}
_rtg_ = rt.interactiveGraphLayout(df, _params_, w=300, h=200)
_rtg_

In [ ]:
_rtg_.selectEntities(set(['test','more','a']))

In [ ]:
_rtg_.selectedEntities()

In [ ]:
set(_rtg_.graphs[0].nodes())

In [ ]:
_rtg_.saveLayout('test_layout.csv')

In [ ]:
_rtg_.loadLayout('test_layout.csv')